In [ ]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# Define the search tool
@tool
def search(query: str):
    """Simulated web search."""
    if "sf" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

tools = [search]
tool_node = ToolNode(tools)

# Initialize the OpenAI model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

# Function to determine whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    return END if not state['messages'][-1].tool_calls else "tools"

# source: https://langchain-ai.github.io/langgraph/#step-by-step-breakdown
# Function to call the model
def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {"messages": [response]}

# Define the workflow
# we initialize graph (StateGraph) by passing state schema (in our case MessagesState)
# MessagesState is a prebuilt state schema that has one attribute -- 
# a list of LangChain Message objects, as well as logic for merging the updates from each node into the state
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.set_entry_point("agent")

workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# Initialize memory to persist state
checkpointer = MemorySaver()
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
    config={"configurable": {"thread_id": 42}}
)
print(final_state["messages"][-1].content)